In [91]:
import os
import time

from web3 import HTTPProvider, Web3

from eth_defi.abi import get_contract
from eth_defi.chain import install_chain_middleware
from eth_defi.event_reader.filter import Filter
from eth_defi.event_reader.reader import read_events, LogResult
from eth_defi.event_reader.reorganisation_monitor import JSONRPCReorganisationMonitor

In [123]:
from eth_defi.abi import get_contract

In [92]:
json_rpc_url = os.environ.get("JSON_RPC_POLYGON", "https://polygon-rpc.com")
web3 = Web3(HTTPProvider(json_rpc_url))
web3.middleware_onion.clear()
install_chain_middleware(web3)

# Get contracts
Pair = get_contract(web3, "sushi/UniswapV2Pair.json")
filter = Filter.create_filter(address=None, event_types=[Pair.events.Swap])  # Listen events from any smart contract

In [93]:
reorg_mon = JSONRPCReorganisationMonitor(web3, check_depth=3)
# Get the headers of last 5 blocks before starting
reorg_mon.load_initial_block_headers(block_count=5)
processed_events = set()
latest_block = reorg_mon.get_last_block_live()
chain_reorg_resolution = reorg_mon.update_chain()
start, end = chain_reorg_resolution.get_read_range()

In [94]:
start = start - 10

In [86]:
from eth_defi.event_reader.conversion import decode_data, convert_uint256_hex_string_to_address, convert_uint256_bytes_to_address, convert_int256_bytes_to_int

In [95]:
events = []

evt: LogResult
for evt in read_events(
            web3,
            start_block=start,
            end_block=end,
            filter=filter,
):
    events.append(evt)
    # How to uniquely identify EVM logs
    key = evt["blockHash"] + evt["transactionHash"] + evt["logIndex"]

    arguments = decode_data(evt["data"])
    topics = evt["topics"]
    token0 = convert_uint256_hex_string_to_address(topics[1])
    token1 = convert_uint256_hex_string_to_address(topics[2])
    pair_address = convert_uint256_bytes_to_address(arguments[0])
    pair_id = convert_int256_bytes_to_int(arguments[1])
    
    # The reader may cause duplicate events as the chain tip reorganises
    if key not in processed_events:
        print(f"Swap at block:{evt['blockNumber']:,} tx:{evt['transactionHash']} pair:{pair_address}")
        processed_events.add(key)
else:
    print(".")

Swap at block:60,234,711 tx:0xbe23aac214c4c4311f8345606cafcddb28e686fb85ee5f4c0b5899d4bfb6c261 pair:0x0000000000000000000000000000000000000000
Swap at block:60,234,711 tx:0x89e3346f92261ea682ce3d334ca975d13e33fdd174b5520429573e20af9faa74 pair:0x0000000000000000000000000000000000000000
Swap at block:60,234,713 tx:0x5efbfa4d428146d2e7050b6a7633d9ae9867e1a8522a2a943bb5ea2717a8512b pair:0x00000000000000000000000000000000046a6775
Swap at block:60,234,715 tx:0xdf20bdb3fd334dd0aae5ceeb9b3496218244e0b905cc6ce988d8ebf78ddbad45 pair:0x000000000000000000000000000000000362b9Ae
Swap at block:60,234,716 tx:0x0cba3b767c0dc84c77fa46b6a9138b06c59426da5c6a45e1b0311cc9922e0ace pair:0x0000000000000000000000000000000000000000
Swap at block:60,234,716 tx:0xe9938c4498896409c78fe5d5c8d2fcf17a64801587db7e58ba9a8363c41c9a30 pair:0x00000000000000000000000028F72DB0F216A7E5
Swap at block:60,234,716 tx:0xe9938c4498896409c78fe5d5c8d2fcf17a64801587db7e58ba9a8363c41c9a30 pair:0x00000000000000000000000083fc4C26045b3157

In [122]:
evt = events[2]

arguments = decode_data(evt["data"])
topics = evt["topics"]

event = {}
event['chain'] = 'polygon'
event['address'] = evt["address"]
event['tx_hash'] = evt["transactionHash"]
event['blk_num'] = evt["blockNumber"]
event['timestamp'] = evt["timestamp"]
event['event_type'] = evt["event"]
event['event_details'] = {}
event['event_details']['token0'] = convert_uint256_hex_string_to_address(topics[1])
event['event_details']['token1'] = convert_uint256_hex_string_to_address(topics[2])
event['event_details']['amount0In'] = convert_int256_bytes_to_int(arguments[0]) 
event['event_details']['amount1Out'] = convert_int256_bytes_to_int(arguments[3]) 

event

{'chain': 'polygon',
 'address': '0x78fd833ec4464d3d6d470e5853dbfc2de0ca6f5b',
 'tx_hash': '0x5efbfa4d428146d2e7050b6a7633d9ae9867e1a8522a2a943bb5ea2717a8512b',
 'blk_num': 60234713,
 'timestamp': 1722887835,
 'event_type': web3._utils.datatypes.Swap,
 'event_details': {'token0': '0x8308a7D7655C2ae21632eD3dd6F55f509E7dfDb9',
  'token1': '0xE2971f917dDC66677668a4b0d14a27Fd569133f7',
  'amount0In': 74082165,
  'amount1Out': 13502000000000000000000}}